https://docs.langchain.com/oss/python/langchain/knowledge-base

##Importante

Percebi que com textos em portguês não foi possível localizar o conteúdo por nenhum dos métodos do langchain-core. preciso entender se o que aconteceu. Criei outro arquivo com pdf em inglês e perguntas também e inglês e funcionou bem.

In [ ]:
pip install langchain-community pypdf langchain-openai ipywidgets langchain-core -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

apikeyGpt = os.environ.get('LANGSMITH_API_KEY')
if not apikeyGpt:
    raise RuntimeError("LANGSMITH_API_KEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = apikeyGpt


In [3]:
## Loading documents
from langchain_community.document_loaders import PyPDFLoader

file_path = "./arquivos/manual_avancado_nobreak_fxp2000.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

print('Paginas:' +  str(len(docs)))
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)


Paginas:3
Manual Técnico - Nobreak FXP-2000
1. Introdução
O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos
sensíveis. Ideal para ambientes domésticos e corporati

{'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250412132946', 'source': './arquivos/manual_avancado_nobreak_fxp2000.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [4]:
##Splitting documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

print(f"{all_splits[3].page_content}")

5
profissional habilitado. Nunca exponha as baterias ao calor ou fogo. Descarte em centros autorizados
conforme regulamentação ambiental.
9. Especificações Adicionais
- Eficiência: até 95%
- Autonomia típica: 25 minutos a 50% da carga
- Tempo de recarga: até 90% em 8 horas
- Interface de monitoramento: USB, Wi-Fi
- Proteções: curto-circuito, surtos, sobretemperatura, inversão de polaridade


In [5]:
apikeyGpt = os.environ.get('GPT_APIKEY')
if not apikeyGpt:
    raise RuntimeError("GPT_APIKEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["OPENAI_API_KEY"] = apikeyGpt

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)
embeddings.embed_query(all_splits[2].page_content)
embeddings.embed_query(all_splits[3].page_content)
embeddings.embed_query(all_splits[4].page_content)


assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_2)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[-0.002805858850479126, 0.03303023427724838, 0.001677811611443758, -0.011274133808910847, -0.027022432535886765, 0.024297373369336128, -0.03632565215229988, 0.044690944254398346, -0.011356519535183907, 0.05647840350866318]


In [6]:
## Creating a vector store InMemory
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# indexing the document splits
ids = vector_store.add_documents(documents=all_splits)

In [7]:
# performing a similarity search
results = vector_store.similarity_search(
 ##   "Quero usar o nobreak FXP2000 em equipamentos hospitalares. Posso fazer isso?"
 "Qual a especificação de autonomia do nobreak FXP2000 com 50% de carga?"
)

print(results[0])

page_content='Manual Técnico - Nobreak FXP-2000
1. Introdução
O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos
sensíveis. Ideal para ambientes domésticos e corporativos, seu sistema inteligente de gerenciamento de
energia assegura operação estável mesmo em condições adversas da rede elétrica.
2. Aplicações Recomendadas
Equipamentos compatíveis incluem: computadores de alta performance, consoles de videogame,
roteadores, servidores compactos, dispositivos de rede, TVs e sistemas de automação residencial. Não
recomendado para uso com equipamentos hospitalares ou industriais pesados.
3. Instalação
Antes da instalação, verifique a tensão da rede elétrica. Posicione o nobreak em local seco, com ventilação
mínima de 10 cm nas laterais. Conecte os dispositivos às saídas traseiras, respeitando o limite de potência
total. A conexão ao aterramento é obrigatória e deve seguir a norma NBR 5410.
4. Características Técnicas
- Potência nominal: 2000VA'

In [8]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score( "Qual a especificação de autonomia do nobreak FXP2000 com 50% de carga?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7020577405963678

page_content='Manual Técnico - Nobreak FXP-2000
1. Introdução
O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos
sensíveis. Ideal para ambientes domésticos e corporativos, seu sistema inteligente de gerenciamento de
energia assegura operação estável mesmo em condições adversas da rede elétrica.
2. Aplicações Recomendadas
Equipamentos compatíveis incluem: computadores de alta performance, consoles de videogame,
roteadores, servidores compactos, dispositivos de rede, TVs e sistemas de automação residencial. Não
recomendado para uso com equipamentos hospitalares ou industriais pesados.
3. Instalação
Antes da instalação, verifique a tensão da rede elétrica. Posicione o nobreak em local seco, com ventilação
mínima de 10 cm nas laterais. Conecte os dispositivos às saídas traseiras, respeitando o limite de potência
total. A conexão ao aterramento é obrigatória e deve seguir a norma NBR 5410.
4. Características Técnicas


In [9]:
embedding = embeddings.embed_query("Qual a especificação de autonomia do nobreak FXP2000 com 50% de carga?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Manual Técnico - Nobreak FXP-2000
1. Introdução
O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos
sensíveis. Ideal para ambientes domésticos e corporativos, seu sistema inteligente de gerenciamento de
energia assegura operação estável mesmo em condições adversas da rede elétrica.
2. Aplicações Recomendadas
Equipamentos compatíveis incluem: computadores de alta performance, consoles de videogame,
roteadores, servidores compactos, dispositivos de rede, TVs e sistemas de automação residencial. Não
recomendado para uso com equipamentos hospitalares ou industriais pesados.
3. Instalação
Antes da instalação, verifique a tensão da rede elétrica. Posicione o nobreak em local seco, com ventilação
mínima de 10 cm nas laterais. Conecte os dispositivos às saídas traseiras, respeitando o limite de potência
total. A conexão ao aterramento é obrigatória e deve seguir a norma NBR 5410.
4. Características Técnicas
- Potência nominal: 2000VA'